In [48]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
import pandas as pd
import xgboost
import os
import pickle

In [49]:
df_2015 = pd.read_csv("../2015_baby/data/master.csv")
df_2016 = pd.read_csv("../2016_baby/data/master.csv")
df_2017 = pd.read_csv("../2017_baby/data/master.csv")
df_2018 = pd.read_csv("../2018_baby/data/master.csv")
df_2014 = pd.read_csv("../2014_baby/data/master.csv")

In [50]:
df_master = df_2015.append(df_2016)
df_master = df_master.append(df_2017)
df_master = df_master.append(df_2018)
df_master = df_master.append(df_2014)
df_master = df_master.reset_index()
df_master = df_master.drop(["index"], axis=1)

In [51]:
non_stats_columns = ["name_x", "date", "name_y", "opp_name_x"]
stats_columns = ["assists_game_x", "blocks_game_x", "points_game_x", "total_rebounds_game_x", "assists_game_y", "blocks_game_y", "points_game_y", "total_rebounds_game_y", "outcomes_game_x"]

df_test = df_master.drop(non_stats_columns, axis=1)
df_test = df_test.drop(stats_columns, axis=1)

In [52]:
stats_columns = ["assists_game_x", "blocks_game_x", "points_game_x", "total_rebounds_game_x", "assists_game_y", "blocks_game_y", "points_game_y", "total_rebounds_game_y"]
y_list = []
for column in stats_columns:
    with open("model_pickle_" + column  + ".pickle", "rb") as pickle_file:
        model = pickle.load(pickle_file)
    
    y_pred = model.predict(df_test)
    y_list.append(y_pred)
    print("done")

[01:09:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
done
[01:09:12] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
done
[01:09:13] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
done
[01:09:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
done
[01:09:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
done
[01:09:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squar

In [55]:
counter = 0
for item in y_list:
    df_test[stats_columns[counter] + "_preds"] = item
    counter = counter+1
df_test

,status_x,assists_avg_x,att_field_goals_avg_x,att_free_throws_avg_x,att_three_points_avg_x,blocks_avg_x,defensive_rebounds_avg_x,made_field_goals_avg_x,made_free_throws_avg_x,made_three_point_avg_x,...,total_rebounds_10MA_y,opp_total_rebounds_10MA_y,assists_game_x_preds,blocks_game_x_preds,points_game_x_preds,total_rebounds_game_x_preds,assists_game_y_preds,blocks_game_y_preds,points_game_y_preds,total_rebounds_game_y_preds
0,0,26.666667,82.916667,19.416667,24.916667,4.916667,31.833333,39.083333,15.666667,8.916667,...,45.0,43.3,25.282455,6.322003,101.643997,41.041595,23.578323,3.500827,95.562981,44.934196
1,1,26.307692,82.384615,19.000000,25.230769,5.076923,32.153846,38.615385,15.230769,9.153846,...,44.1,43.5,27.139687,5.978418,107.243454,41.938000,24.504448,2.111984,99.973793,44.469936
2,0,26.071429,82.857143,19.357143,25.571429,5.142857,32.214286,38.571429,15.428571,9.142857,...,45.4,40.4,22.705105,4.145417,97.511284,39.686298,24.663622,6.724072,102.119537,47.067951
3,1,25.800000,83.066667,19.733333,25.733333,5.066667,31.800000,38.133333,15.866667,9.266667,...,44.8,42.7,26.227039,6.542183,103.838974,42.587337,25.209715,2.703465,100.615578,44.765110
4,0,26.250000,82.875000,19.875000,25.625000,5.187500,32.125000,38.562500,16.062500,9.437500,...,45.7,40.9,22.519844,6.494698,101.948532,40.607101,22.983583,4.181939,100.880814,44.839066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11551,0,24.045977,82.862069,21.770115,17.275862,4.689655,34.356322,38.333333,16.160920,6.310345,...,44.3,46.1,23.662294,2.672907,97.868973,45.838776,24.697680,5.464766,101.937134,42.237751
11552,1,24.011364,82.852273,21.681818,17.329545,4.647727,34.340909,38.295455,16.068182,6.375000,...,44.9,45.0,24.093613,4.910276,99.149345,46.762398,22.782799,4.168169,99.614029,41.044247
11553,1,24.044944,82.797753,21.764045,17.449438,4.651685,34.337079,38.280899,16.101124,6.415730,...,45.0,45.4,24.606396,5.928933,98.970840,46.747131,22.661957,4.883868,99.390343,41.236889
11554,0,24.011111,82.822222,21.711111,17.544444,4.688889,34.300000,38.277778,16.066667,6.477778,...,44.5,44.5,23.214136,4.499959,96.895409,45.453869,25.157864,8.184999,102.440582,42.607731


In [56]:
x_train = df_test

stats_columns = ["outcomes_game_x"]
for column in stats_columns:
    # Step 1
    params = {
        "n_estimators": [400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]
    }
    classifier = xgboost.XGBRegressor()
    
    y_train = df_master[column]
    random_search = RandomizedSearchCV(classifier,param_distributions=params,scoring="neg_mean_absolute_error",n_iter=13,n_jobs=-1,cv=5,verbose=3)
    random_search.fit(x_train, y_train)
    best = random_search.best_params_

    classifier = xgboost.XGBRegressor(n_estimators=best["n_estimators"])

    # step 2
    params = {
        "max_depth": [3, 4, 5, 6, 7, 9, 10, 12]
    }

    random_search = RandomizedSearchCV(classifier,param_distributions=params,scoring="neg_mean_absolute_error",n_iter=8,n_jobs=-1,cv=5,verbose=3)
    random_search.fit(x_train, y_train)
    best_test = random_search.best_params_

    best["max_depth"] = best_test["max_depth"]
    classifier = xgboost.XGBRegressor(n_estimators=best["n_estimators"], max_depth = best["max_depth"])
    
    
    # Step 3
    params = {
        "learning_rate": [0.01, 0.03, 0.05, 0.07, 0.09, 0.1],
        "subsample": [0.8, 0.85, 0.9, 0.95, 1],
        "gamma": [0, 1, 5,],
        "colsample_bytree": [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    }

    random_search = RandomizedSearchCV(classifier,param_distributions=params,scoring="neg_mean_absolute_error",n_iter=50,n_jobs=-1,cv=5,verbose=3)
    random_search.fit(x_train, y_train)
    best_test = random_search.best_params_

    best["learning_rate"] = best_test["learning_rate"]
    best["subsample"] = best_test["subsample"]
    best["gamma"] = best_test["gamma"]
    best["colsample_bytree"] = best_test["colsample_bytree"]

    classifier = xgboost.XGBRegressor(n_estimators=best["n_estimators"], max_depth = best["max_depth"], learning_rate = best["learning_rate"], subsample=best["subsample"], gamma=best["gamma"], colsample_bytree=best["colsample_bytree"])
    
    classifier.fit(x_train,y_train)

    with open('model_pickle_' + column + '.pickle', 'wb') as f:
        pickle.dump(classifier, f)

Fitting 5 folds for each of 13 candidates, totalling 65 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed: 24.7min finished


[01:35:00] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 19.5min finished


[01:55:18] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 33.5min finished


[02:29:42] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:30:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
